In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

from src.accelerometer import plot_fourier_transformation, plot_acceleration, plot_feature_columns, accelerometer_feature_engineering
from src.kmeans import kmeans
from src.decision_tree import decision_tree
from src.plotting import box_plot_columns
from src.time_series import median_filter, run_time_series_algorithms
from src.ml_util import run_feature_algorithms
from src.pandas_util import correlation_matrix,extract_sample_sessions,extract_subject_dataframes,reduce_to_one_session_for_subject

In [ ]:
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

In [ ]:
slow_move_df = pd.read_csv('slow_move_accelerations.csv')
slow_move_df = slow_move_df[slow_move_df['age_group']!=0]

slow_move_touches_df = pd.read_csv('slow_move_touches.csv')
slow_move_touches_df = slow_move_touches_df[slow_move_touches_df['age_group']!=0]

In [ ]:
slow_move_df.head(5)

In [ ]:
slow_move_touches_df.head()

In [ ]:
slow_move_df.describe()

In [ ]:
slow_move_df = median_filter(slow_move_df)

In [ ]:
single_session_30_df, single_session_50_df = extract_sample_sessions(slow_move_df)
plot_acceleration(single_session_30_df, title='age-group \'<30\'', subplots=False)
plot_acceleration(single_session_50_df, title='age-group \'>50\'', subplots=False)

In [ ]:
subject_30_df, subject_50_df = extract_subject_dataframes(slow_move_df)
plot_acceleration(subject_30_df, title='age-group \'<30\'', subplots=False)
plot_acceleration(subject_50_df, title='age-group \'>50\'', subplots=False)

In [ ]:
sessions_30_df = slow_move_df[slow_move_df['age_group']==30]
sessions_50_df = slow_move_df[slow_move_df['age_group']==50]
plot_acceleration(reduce_to_one_session_for_subject(sessions_30_df),title='age-group \'<30\'',subplots=True)
plot_acceleration(reduce_to_one_session_for_subject(sessions_50_df),title='age-group \'>50\'',subplots=True)

In [ ]:
plot_fourier_transformation(single_session_30_df, 'test session')

In [ ]:
box_plot_columns(single_session_30_df)

In [ ]:
plot_acceleration(single_session_30_df)

In [ ]:
slow_move_processed_df = accelerometer_feature_engineering(slow_move_df)
slow_move_processed_df = slow_move_processed_df.reset_index(drop=False)

In [ ]:
slow_move_touches_df['center_line_deviation'] = np.abs(slow_move_touches_df['y'] - slow_move_touches_df['start_y'])

In [ ]:
slow_move_processed_df['y_touch_std'] =  np.nan
for uuid in slow_move_touches_df['uuid'].unique():
    current_session = slow_move_touches_df[slow_move_touches_df['uuid']==uuid]
    y_touch_std = current_session['x'].std()
    slow_move_processed_df.loc[slow_move_processed_df['uuid'] == uuid, 'y_touch_std'] = y_touch_std
    slow_move_processed_df.loc[slow_move_processed_df['uuid'] == uuid, 'center_line_deviation'] = current_session['center_line_deviation'].mean()

In [ ]:
slow_move_processed_df.head()

In [ ]:
correlation_matrix(slow_move_processed_df)

SEM is closely related to std -> therefore we drop it

In [ ]:
slow_move_processed_df = slow_move_processed_df.drop(columns=['mag_sem','x_snr','y_snr','z_snr','mag_snr','x_peaks','y_peaks','z_peaks'])

In [ ]:
plot_feature_columns(slow_move_processed_df,'std')
plot_feature_columns(slow_move_processed_df,'mean')
plot_feature_columns(slow_move_processed_df,'sal')
box_plot_columns(slow_move_processed_df, 'age_group',['x_sem','y_sem','z_sem'])
box_plot_columns(slow_move_processed_df, 'age_group',['mag_peaks'])
box_plot_columns(slow_move_processed_df, 'age_group',['duration'])
box_plot_columns(slow_move_processed_df, 'age_group',['y_touch_std'])
box_plot_columns(slow_move_processed_df, 'age_group',['center_line_deviation'])

We can see that there is no significant deviation between the two age groups in the following features:
- y_mean
- mag_mean
- z_snr
- mag_snr
- z_sal
- mag_sal

# ML Models

In [ ]:
results = {} 

In [ ]:
feature_keys = ['x_std','y_std','x_mean','x_sem','hand']
class_key = ['age_group']

In [ ]:
slow_move_processed_df = slow_move_processed_df.sort_values(by='age_group')
for feature in feature_keys:
    box_plot_columns(slow_move_processed_df, 'subject',[feature],show_column_in_label=False)

In [ ]:
slow_move_processed_df['age_group'].replace(to_replace=30,value=1,inplace=True)
slow_move_processed_df['age_group'].replace(to_replace=50,value=0,inplace=True)
slow_move_processed_df['hand'].replace(to_replace='dominant',value=1,inplace=True)
slow_move_processed_df['hand'].replace(to_replace='non_dominant',value=0,inplace=True)

## Run Algorithms on extracted Features

In [ ]:
feature_results = run_feature_algorithms(slow_move_processed_df, feature_keys)
results.update(feature_results)

## Run Time Series Algorithm

In [ ]:
time_series_results = run_time_series_algorithms(slow_move_df)
results.update(time_series_results)

## ML Results

In [ ]:
results